Jupyter notebook to access star clusters, compact associations and other noise in nearby galaxies. This notebook only downloads necessary files locally. Images are accessed in access_images.ipynb to cut down on execution time. 

The data release is presented here:
https://archive.stsci.edu/hlsp/phangs/phangs-cat
and the accompanying paper is [Maschmann et al. (2024)](https://ui.adsabs.harvard.edu/abs/2024arXiv240304901M/abstract).

This notebook was adapted from [Maschmann et al. (2024)]https://github.com/DanielMaschmann/PHANGS_HST_star_cluster/ 


In [3]:
# packages needed to for file management,
# downloading and unpacking compressed catalogs.
import os
from urllib3 import PoolManager
import tarfile

# basic packages for data handling and plotting
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import random

from astropy.io import fits
from astropy.table import Table
import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
from astropy.stats import sigma_clipped_stats

from scipy.constants import c as speed_of_light

# we download a file with multiple helpfull methods from the same git repository.
# These functions are well documented to facilitate further reading.
helper_func_url = 'https://raw.githubusercontent.com/DanielMaschmann/PHANGS_HST_star_cluster/main/helper_func.py'
# Download file. This will save them in the working directory.
http = PoolManager()
http_request_object = http.request('GET', helper_func_url, preload_content=False)
with open('helper_func.py', 'wb') as out:
    while True:
        data = http_request_object.read()
        if not data:
            break
        out.write(data)

!pip install dust_extinction

# in case of updates
%load_ext autoreload
%autoreload 2
import importlib
import helper_func as helper_func
mylibrary = importlib.reload(helper_func)

In this Jupyter Notebook we will demonstrate to access and handle the cluster catalog for the Galaxy NGC 1566

In [4]:
# we use NGC ngc1097 as an example
galaxy_name = 'ngc1097'


Download the cluster and compact associataion catalogs and extract them.
The catalog data release is described here https://archive.stsci.edu/hlsp/phangs/phangs-cat and the list of the data products can be found here: https://archive.stsci.edu/hlsp/phangs.html#hst_image_products_table \

In [5]:
# get the file name and the URL for the catalogs
url_path_structure = 'https://archive.stsci.edu/hlsps/phangs-cat/dr4/bundles/'
catalog_file_name_compressed = ('hlsp_phangs-cat_hst_acs-uvis_%s_multi_v1_cats.tar.gz' %
                    galaxy_name)
url = url_path_structure + catalog_file_name_compressed
# Download compressed catalogs. This will save them in the working directory.
http = PoolManager()
http_request_object = http.request('GET', url, preload_content=False)
with open(catalog_file_name_compressed, 'wb') as out:
    while True:
        data = http_request_object.read()
        if not data:
            break
        out.write(data)
# get the present work directory (pwd) where we will extract the file
pwd = os.getcwd()
src_path = pwd + '/' + catalog_file_name_compressed
# extract the compressed files locally
tar = tarfile.open(src_path, 'r:gz')
tar.extractall(pwd)
tar.close()

In [6]:
# some galaxies have HST image mosaics. Therefore the file names can change!
mosaic_flag = True
if mosaic_flag:
  bundle_file_name = galaxy_name + 'mosaic'
else:
  bundle_file_name = galaxy_name

In [7]:
img_file_name_compressed = 'hlsp_phangs-hst_hst_wfc3-uvis_%s_multi_v1_drc-bundle.tar.gz' % bundle_file_name
url = 'https://archive.stsci.edu/hlsps/phangs-hst/' + bundle_file_name + '/' + img_file_name_compressed
# Download compressed image bundle. This will save them in the working directory.
http = PoolManager()
http_request_object = http.request('GET', url, preload_content=False)
with open(img_file_name_compressed, 'wb') as out:
    while True:
        data = http_request_object.read()
        if not data:
            break
        out.write(data)
# get the present work directory (pwd) where we will extract the file
pwd = os.getcwd()
src_path = pwd + '/' + img_file_name_compressed
# extract the compressed files locally
tar = tarfile.open(src_path, 'r:gz')
tar.extractall(pwd)
tar.close()